In [151]:
import pandas as pd
import numpy as np
import altair as alt
import statsmodels.formula.api as smf
import pathlib

In [152]:
data = pd.read_parquet(f"{pathlib.Path.cwd()}/../20_intermediate_files/Death_and_Population.gzip")
data.head()

,County Code,Year,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,target_state,State,County,Population,_merge
index,,,,,,,,,,
0,4001.0,2003.0,All other alcohol-induced causes,A9,22.0,1,AZ,Apache County,68072.0,both
1,4001.0,2003.0,All other non-drug and non-alcohol causes,O9,464.0,1,AZ,Apache County,68072.0,both
2,4003.0,2003.0,Drug poisonings (overdose) Unintentional (X40-...,D1,11.0,1,AZ,Cochise County,120638.0,both
3,4003.0,2003.0,All other alcohol-induced causes,A9,14.0,1,AZ,Cochise County,120638.0,both
4,4003.0,2003.0,All other non-drug and non-alcohol causes,O9,1109.0,1,AZ,Cochise County,120638.0,both


In [153]:
## Calculate overdose death per capta
data["Overdose_death_per_capita"] = data["Deaths"] / data["Population"]
data.head()

,County Code,Year,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,target_state,State,County,Population,_merge,Overdose_death_per_capita
index,,,,,,,,,,,
0,4001.0,2003.0,All other alcohol-induced causes,A9,22.0,1,AZ,Apache County,68072.0,both,0.000323
1,4001.0,2003.0,All other non-drug and non-alcohol causes,O9,464.0,1,AZ,Apache County,68072.0,both,0.006816
2,4003.0,2003.0,Drug poisonings (overdose) Unintentional (X40-...,D1,11.0,1,AZ,Cochise County,120638.0,both,0.000091
3,4003.0,2003.0,All other alcohol-induced causes,A9,14.0,1,AZ,Cochise County,120638.0,both,0.000116
4,4003.0,2003.0,All other non-drug and non-alcohol causes,O9,1109.0,1,AZ,Cochise County,120638.0,both,0.009193


In [154]:
# Select target states
TX = data[data.State == "TX"] # Texas
WI = data[data.State == "WI"] # Wisconsin
MS = data[data.State == "MS"] # Mississippi
KS = data[data.State == "KS"] # Kansus

In [155]:
# For Texas, the policy was effective since 2007, split data
TX_before = TX[TX["Year"] < 2007]
TX_after = TX[TX["Year"] >= 2007]

In [156]:
# Mark a vertical line at a given year
def get_vertical_line(year):
    line = alt.Chart(pd.DataFrame({
    "Year": [year]
    })).mark_rule(
        color="red"
    ).encode(
    x="Year:Q"
    )
    return line

In [157]:
mark_vertical_line(2007).display()

alt.Chart(...)

In [158]:
# Get a grid for future prediction
def get_plot_grid(df, x, y):
    X = df.loc[pd.notnull(df[y]), x]
    X_min = X.min()
    X_max = X.max()
    X_step = (X_max - X_min) / 100
    return np.arange(X_min, X_max + X_step, X_step)

In [159]:
TX_before.head()

,County Code,Year,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,target_state,State,County,Population,_merge,Overdose_death_per_capita
index,,,,,,,,,,,
490,48001.0,2003.0,All other non-drug and non-alcohol causes,O9,583.0,1,TX,Anderson County,56068.0,both,0.010398
491,48003.0,2003.0,All other non-drug and non-alcohol causes,O9,105.0,1,TX,Andrews County,12976.0,both,0.008092
492,48005.0,2003.0,All other non-drug and non-alcohol causes,O9,784.0,1,TX,Angelina County,81510.0,both,0.009618
493,48007.0,2003.0,All other non-drug and non-alcohol causes,O9,255.0,1,TX,Aransas County,22843.0,both,0.011163
494,48009.0,2003.0,All other non-drug and non-alcohol causes,O9,66.0,1,TX,Archer County,9013.0,both,0.007323


In [160]:
x = "Year"
y = "Overdose_death_per_capita"
TX_before_grid = get_plot_grid(TX_before, x, y)

In [161]:
def get_ols_prediction(df, grid, x, y):
    predictions = pd.DataFrame({x: grid})
    model = smf.ols(f"{y} ~ {x}", data=df).fit()
    model_predict = model.get_prediction(predictions[x])
    predictions[y] = model_predict.summary_frame()["mean"]
    predictions[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=0.05)
    return predictions

In [162]:
TX_before_predictions = get_ols_prediction(TX_before, TX_before_grid, x, y)
TX_before_predictions.head()

,Year,Overdose_death_per_capita,ci_low,ci_high
0,2003.00,0.008230,0.007772,0.008689
1,2003.03,0.008225,0.007772,0.008677
2,2003.06,0.008219,0.007773,0.008666
3,2003.09,0.008214,0.007773,0.008655
4,2003.12,0.008208,0.007773,0.008643


In [163]:
def get_regression_chart(df, x, y):
    regression_chart = (
        alt.Chart(df)
        .mark_line()
        .encode(x=x, y=alt.Y(y, scale=alt.Scale(zero=False)))
    )
    return regression_chart

In [164]:
def get_ci_chart(df, x, y):
    ci_chart = (
        alt.Chart(df)
        .mark_errorband()
        .encode(
            x=x,
            y=alt.Y("ci_low", title=y),
            y2="ci_high",
        )
    )
    return ci_chart


In [165]:
TX_before_reg_chart = get_regression_chart(TX_before_predictions, x, y)
TX_before_ci_chart = get_ci_chart(TX_before_predictions, x, y)

res = TX_before_reg_chart + TX_before_ci_chart
res.display()

alt.LayerChart(...)

In [166]:
def pre_post_analysis(dataframe, year):
    x = "Year"
    y = "Overdose_death_per_capita"
    before = dataframe[dataframe["Year"] < year]
    after = dataframe[dataframe["Year"] >= year]
    
    before_grid = get_plot_grid(before, x, y)
    before_predictions = get_ols_prediction(before, before_grid, x, y)
    before_reg_chart = get_regression_chart(before_predictions, x, y)
    before_ci_chart = get_ci_chart(before_predictions, x, y)

    after_grid = get_plot_grid(after, x, y)
    after_predictions = get_ols_prediction(after, after_grid, x, y)
    after_reg_chart = get_regression_chart(after_predictions, x, y)
    after_ci_chart = get_ci_chart(after_predictions, x, y)
    return before_reg_chart + before_ci_chart, after_reg_chart + after_ci_chart

In [167]:
TX_pre, TX_post = pre_post_analysis(TX, 2007)
TX_pre_post_chart = alt.layer(TX_pre, TX_post, get_vertical_line(2007)).properties()
TX_pre_post_chart.display()

alt.LayerChart(...)

In [168]:
WI_pre, WI_post = pre_post_analysis(WI, 2007)
MS_pre, MS_post = pre_post_analysis(MS, 2007)
KS_pre, KS_post = pre_post_analysis(KS, 2007)
line = get_vertical_line(2007)
TX_vs_WI = alt.layer(TX_pre_post_chart, WI_pre + WI_post, line).properties(
    title="Overdose Death Rates in Texas vs. Wisconsin"
)
TX_vs_MS = alt.layer(TX_pre_post_chart, MS_pre + MS_post, line).properties(
    title="Overdose Death Rates in Texas vs. Mississippi"
)
TX_vs_KS = alt.layer(TX_pre_post_chart, KS_pre + KS_post, line).properties(
    title="Overdose Death Rates in Texas vs. Kansas"
)
TX_vs_KS.display()


alt.LayerChart(...)